# Notebook 3: Conexión a Bases de Datos

---

En este notebook exploraremos cómo conectar y trabajar con bases de datos en Python utilizando el módulo **`pyodbc`**. Aprenderemos a gestionar sesiones de conexión, ejecutar consultas SQL y manejar errores comunes que puedan surgir en el proceso.

---

# ¡IMPORTANTE!: 

Como estamos en un jupyter notebook y no en un directorio como normalmente trabajamos la manera de importar los modulos y de ejecutar los test es distinta por lo que simularemos un flujo de trabajo como si fuera un directorio con scripts estructurados en el que normalmente creamos scripts con los metodos y otro con los test. En jupyter esto no se puede hacer asi que habra ambas implementaciones en el código con sus respectivos comentarios para entender el contexto simulado 

---
## 1. Uso de `pyodbc` para Interactuar con Bases de Datos

**`pyodbc`** es un módulo que permite interactuar con bases de datos mediante el protocolo ODBC (Open Database Connectivity). Este módulo es versátil y compatible con múltiples sistemas de bases de datos como MySQL, SQL Server, SQLite, y otros.

---


### 1.1 Estructura Básica de una Conexión con `pyodbc`




In [ ]:
import pyodbc # Importamos la librería pyodbc para conectarnos a la base de datos (La veremos en otro notebook)

# Creamos la clase SesionDB que se conecta a la base de datos
class SesionDB:
    '''
    Definimos el método __init__ que recibe la url de la base de datos
    Inicializamos la variable conexion en None indicando que la conexión a la base de datos aún no se ha establecido. 
    Aquí hay una explicación más detallada:
        -Estado inicial: Al inicializar en None, se establece un estado inicial claro para la instancia de la clase.
         Esto indica que no hay una conexión activa a la base de datos cuando se crea el objeto.

        -Control de flujo: 
            En el método ejecutar_consulta, se verifica si self.conexion es None antes de intentar ejecutar una consulta. 
            Si es None, se llama al método connect para establecer la conexión. 
            Esto asegura que siempre haya una conexión activa antes de ejecutar cualquier consulta.

        -Manejo de recursos: 
            Inicializar self.conexion en None permite un manejo más sencillo de los recursos. 
            La conexión solo se establece cuando es necesaria, y se puede cerrar explícitamente con el método disconnect.
    '''
    def __init__(self, url):
        self.url = url
        self.conexion = None

    # Definimos el método connect que establece la conexión a la base de datos
    def connect(self):
        self.conexion = pyodbc.connect(self.url) #pyodbc.connect() se utiliza para establecer la conexión a la base de datos
        return self.conexion

    # Definimos el método ejecutar_consulta que recibe una consulta
    def ejecutar_consulta(self, query): #query es la consulta que se va a ejecutar
        # Verificamos si la conexión está establecida
        if self.conexion is None:
            self.connect() # Si la conexión no está establecida, llamamos al método connect para establecerla
        # Creamos un cursor para ejecutar la consulta
        cursor = self.conexion.cursor() #Un cursor es un objeto que permite ejecutar consultas SQL en la base de datos
        cursor.execute(query) # Ejecutamos la consulta que habremos definido en query
        return cursor.fetchall() # fetchall() obtiene todos los resultados de la consulta

    # Definimos el método disconnect que cierra la conexión a la base de datos
    def disconnect(self):
        if self.conexion:
            self.conexion.close() # close() se utiliza para cerrar la conexión a la base de datos

### EJERCICIO:

## 2. Gestión de Sesiones y Ejecución de Consultas

La gestión de sesiones es un aspecto crucial al trabajar con bases de datos. Mantener conexiones abiertas innecesariamente puede llevar a un consumo excesivo de recursos, mientras que abrir y cerrar conexiones repetidamente puede ser costoso en términos de rendimiento. Por eso, una clase como **`SesionDB`** proporciona un enfoque estructurado para manejar estas operaciones de manera eficiente y segura.

---

### 2.1 Clase `SesionDB`: Gestión Encapsulada

La clase `SesionDB` centraliza las operaciones básicas de una conexión:
- **Establecimiento de conexión**: Se conecta a la base de datos solo cuando es necesario.
- **Ejecución de consultas**: Abstrae la lógica de ejecución y manejo de errores.
- **Desconexión**: Garantiza el cierre de la conexión incluso si ocurre un error.



In [ ]:
'''
Configuración de la cadena de conexión a la base de datos SQL Server
Formato: "DRIVER={SQL Server};SERVER=servidor;DATABASE=base_datos;UID=usuario;PWD=contraseña"
    - DRIVER: Especifica el controlador de SQL Server
    - SERVER: Nombre o dirección IP del servidor (localhost para servidor local)
    - DATABASE: Nombre de la base de datos a conectar
    - UID: Nombre de usuario para autenticación
    - PWD: Contraseña del usuario
'''

url = "DRIVER={SQL Server};SERVER=localhost;DATABASE=mi_base;UID=usuario;PWD=contraseña"

# Crear una instancia de la clase SesionDB que maneja la conexión
# Esta clase debe contener los métodos connect(), disconnect() y ejecutar_consulta()
sesion = SesionDB(url)

try:
    # Bloque try-except para manejar posibles errores de conexión y consultas
    
    # Intentar establecer la conexión a la base de datos
    print("Estableciendo conexión...")
    conexion = sesion.connect()
    print("Conexión exitosa.")

    # Ejecutar una consulta de ejemplo
    # TOP 5 limita los resultados a las primeras 5 filas de la tabla
    query = "SELECT TOP 5 * FROM mi_tabla"
    resultados = sesion.ejecutar_consulta(query)
    
    # Iterar sobre los resultados y mostrarlos en consola
    print("Resultados de la consulta:")
    for fila in resultados:
        print(fila)

except pyodbc.Error as e:
    # Capturar y manejar cualquier error específico de pyodbc
    # pyodbc.Error es la clase base para las excepciones de pyodbc
    print("Error al interactuar con la base de datos:", e)

finally:
    # El bloque finally se ejecuta siempre, independientemente de si hubo error o no
    # Asegura que la conexión se cierre adecuadamente para liberar recursos
    sesion.disconnect()
    print("Conexión cerrada.")

#### Explicación del Flujo

1. **Conexión**:
   - `sesion.connect()` establece la conexión utilizando la URL especificada.

2. **Consulta**:
   - La consulta se ejecuta con `sesion.ejecutar_consulta(query)`.
   - El método retorna los resultados como una lista de tuplas.

3. **Desconexión**:
   - `sesion.disconnect()` asegura que la conexión se cierra de forma ordenada para liberar recursos.

4. **Manejo de Errores**:
   - Se captura cualquier error relacionado con la base de datos mediante `pyodbc.Error`.


### 2.2 Buenas Prácticas para Sesiones

1. **Mantén las conexiones abiertas solo mientras sea necesario**.
   - Evita mantener conexiones inactivas abiertas.

2. **Usa parámetros para consultas dinámicas**:
   - Protege contra inyección SQL utilizando `?` en lugar de concatenar valores directamente.
   - Ejemplo:
     ```python
     cursor.execute("SELECT * FROM mi_tabla WHERE id = ?", (valor_id,))
     ```

3. **Loguea errores críticos**:
   - Integra `logging` para registrar excepciones y problemas de conexión.

## 3. Manejo de Errores en Conexiones

El manejo de errores es una parte esencial al trabajar con bases de datos, ya que permite que el programa gestione situaciones imprevistas de manera controlada. Esto incluye errores durante la conexión, consultas inválidas, o problemas con la base de datos.

---

#### Tipos Comunes de Errores en `pyodbc`

1. **Errores de conexión (`pyodbc.InterfaceError`)**:
   - Ocurren si la conexión a la base de datos falla debido a problemas de red, credenciales incorrectas, o configuraciones erróneas del servidor.

2. **Errores de consulta (`pyodbc.ProgrammingError`)**:
   - Generados cuando la consulta SQL tiene un error, como acceso a tablas inexistentes o sintaxis inválida.

3. **Errores generales (`pyodbc.Error`)**:
   - Captura errores no específicos o aquellos que no están cubiertos por las excepciones anteriores.

---

# 3.1 Manejo Básico de Excepciones

Un bloque `try-except` puede capturar estos errores y manejar cada uno de manera específica. Aquí un ejemplo ampliado:

In [ ]:
import pyodbc

# Crear la clase de sesión (ya definida previamente)
class SesionDB:
    def __init__(self, url, conexion=None):
        self.url = url
        self.conexion = conexion

    def connect(self):
        if self.conexion is None:
            self.conexion = pyodbc.connect(self.url)
        return self.conexion

    def ejecutar_consulta(self, query):
        if self.conexion is None:
            self.connect()
        cursor = self.conexion.cursor()
        cursor.execute(query)
        return cursor.fetchall()

    def disconnect(self):
        if self.conexion:
            self.conexion.close()
            self.conexion = None

# Configuración de la URL de conexión
url = "DRIVER={SQL Server};SERVER=localhost;DATABASE=mi_base;UID=usuario;PWD=contraseña"
sesion = SesionDB(url)

# Bloque try-except para manejar errores de conexión y consultas

try: # Intentar establecer la conexión a la base de datos
    print("Intentando conectar...")
    sesion.connect()
    print("Conexión exitosa.")

    # Ejecución de una consulta inválida
    query = "SELECT * FROM tabla_inexistente"
    resultados = sesion.ejecutar_consulta(query)

    print("Resultados de la consulta:")
    for fila in resultados:
        print(fila)

# Manejo de errores específicos
except pyodbc.InterfaceError as e:
    # Manejo de errores de conexión
    print("Error de conexión a la base de datos:", e)

# Manejo de errores generales
except pyodbc.ProgrammingError as e:
    # Manejo de errores de consulta
    print("Error en la consulta SQL:", e)

# Manejo de errores generales
except pyodbc.Error as e:
    # Manejo general de errores
    print("Error general en la base de datos:", e)

finally:
    # Cerrar la conexión en cualquier caso
    sesion.disconnect()
    print("Conexión cerrada.")


#### Explicación del Código

1. **Bloque `try`**:
   - Intenta conectar y ejecutar una consulta.
   - En este caso, la consulta apunta a una tabla inexistente para demostrar el manejo de errores.

2. **Bloques `except`**:
   - **`pyodbc.InterfaceError`**:
     Maneja errores relacionados con la conexión (por ejemplo, servidor inaccesible).
   - **`pyodbc.ProgrammingError`**:
     Captura errores en la consulta, como intentar acceder a una tabla inexistente.
   - **`pyodbc.Error`**:
     Captura cualquier otro error general.

3. **Bloque `finally`**:
   - Garantiza que la conexión se cierre incluso si ocurre un error.

### Ejercicio: Gestión y Manejo de Conexiones a Bases de Datos con `pyodbc`

#### Enunciado:

Eres responsable de desarrollar un sistema de interacción con una base de datos utilizando `pyodbc`. Debes implementar un script que cumpla con los siguientes requisitos:

1. **Establecer una conexión a la base de datos**:
   - Utiliza la clase `SesionDB` para gestionar la conexión.
   - Personaliza la URL de conexión para que funcione con un servidor local o remoto.

2. **Ejecutar consultas**:
   - Implementa una función que reciba como parámetro una consulta SQL y devuelva los resultados.
   - Si la consulta falla (por ejemplo, debido a una tabla inexistente), captura el error y muestra un mensaje informativo.

3. **Registrar errores**:
   - Configura un sistema de logging que:
     - Muestre errores críticos en la consola.
     - Guarde todos los errores en un archivo llamado `errores_bd.log`.

4. **Seguridad y robustez**:
   - Asegúrate de cerrar la conexión a la base de datos al final de la ejecución, incluso si ocurre un error.

5. **Patrón de contexto (`with`)** (opcional):
   - Modifica la clase `SesionDB` para que sea compatible con el uso del patrón `with`, asegurando que las conexiones siempre se cierren al finalizar el bloque.

#### Pasos Específicos:

1. **Preparar la base de datos**:
   - Crea una base de datos con al menos una tabla llamada `empleados` que contenga las siguientes columnas:
     - `id` (entero, clave primaria).
     - `nombre` (texto).
     - `puesto` (texto).
   - Inserta al menos 3 registros de prueba.

2. **Implementar la clase `SesionDB`**:
   - Incluye métodos para conectar, ejecutar consultas y cerrar la conexión.

3. **Ejecutar consultas**:
   - Realiza dos consultas:
     - Una consulta válida para obtener todos los registros de la tabla `empleados`.
     - Una consulta inválida que intente acceder a una tabla inexistente para simular un error.

4. **Registrar errores**:
   - Configura el sistema de logging para capturar y guardar los errores generados por la consulta inválida.

5. **Documentar el flujo**:
   - Explica con comentarios cómo funciona cada parte del código.

---

#### Ejemplo de Salida Esperada:

1. **Consola**:
   ```
   Intentando conectar...
   Conexión exitosa.
   Resultados de la consulta válida:
   (1, 'Juan Pérez', 'Desarrollador')
   (2, 'Ana Gómez', 'Analista')
   (3, 'Luis Martínez', 'Administrador')
   Error en la consulta SQL: ('42S02', "[42S02] [Microsoft][SQL Server Native Client 11.0][SQL Server]Invalid object name 'tabla_inexistente'.")
   Conexión cerrada.
   ```

2. **Archivo `errores_bd.log`**:
   ```
   2024-12-30 20:15:00,123 - ERROR - Error en la consulta SQL: ('42S02', "[42S02] [Microsoft][SQL Server Native Client 11.0][SQL Server]Invalid object name 'tabla_inexistente'.")
   ```

#### Objetivo del Ejercicio:
Este ejercicio te ayudará a consolidar los conceptos aprendidos sobre:
- Conexión a bases de datos.
- Gestión de sesiones.
- Manejo de errores específicos.
- Uso de `logging` para registrar errores de forma eficiente y profesional.
